In [ ]:
# operators.ipynb

# Cell 1 - Create two random state vectors (not normalized)

import numpy as np

# import random
from IPython.display import Math
from qiskit.visualization import array_to_latex

np.random.seed(2016)
n = 5

psi = np.random.random(n) + np.random.random(n) * 1j
phi = np.random.random(n) + np.random.random(n) * 1j

display(array_to_latex(psi[:, np.newaxis], prefix=r"\mathbf{\lvert\psi\rangle}="))
display(array_to_latex(phi[:, np.newaxis], prefix=r"\mathbf{\lvert\phi\rangle}="))

In [ ]:
# Cell 2 - Create a Hermitian Operator (as a matrix)


def create_hermitian_matrix(n):
    a = np.zeros((n, n), dtype=complex)
    for i in range(n):
        for j in range(i, n):
            r1 = np.random.random()
            r2 = np.random.random()
            if i == j:
                a[i, j] = complex(r1, 0)
            else:
                a[i, j] = complex(r1, r2)
                a[j, i] = complex(r1, -r2)
    return a


op = create_hermitian_matrix(n)

display(array_to_latex(op, prefix=r"\mathbf{\hat{O}}="))

In [ ]:
# Cell 3 - A Hermitian operator applied to its eigenkets produce its eigenvalues

eigen_vals, eigen_vecs = np.linalg.eig(op)

# Note: The eigenvalues of a Hermitian operator are all real
display(
    array_to_latex(
        eigen_vals[:, np.newaxis].T,
        prefix=r"\mathbf{\lambda}=",
    )
)

# Note: In numpy, eigenvectors are returned as columns
for i in range(n):
    display(array_to_latex(eigen_vecs[:, i], prefix=rf"\mathbf{{v_{i}}}="))

bra_phi = phi.conj().T

for i in range(n):
    t1 = np.dot(bra_phi, np.dot(op, eigen_vecs[:, i]))
    t2 = np.dot(bra_phi, eigen_vals[i] * eigen_vecs[:, i])
    display(
        Math(
            rf"\mathbf{{\langle\phi\lvert\hat{{O}}\lvert v_{i}\rangle="
            rf"\langle\phi\lvert\lambda_{i}\lvert v_{i}\rangle}}\;?\;\rightarrow\;{np.isclose(t1,t2)}"
        )
    )

In [ ]:
# Cell 4 - All non-degenerate eigenvectors
# of a Hermitian operator are orthogonal to each other

for i in range(n):
    for j in range(i + 1, n):
        display(
            Math(
                rf"\mathbf{{v_{i}\cdot v_{j}}}\;=\;"
                rf"{np.dot(eigen_vecs[:,i].conj(),eigen_vecs[:,j]).round(4)}"
            )
        )

In [ ]:
# Cell 5 - Get Matrix From Operator in a given basis


def get_matrix_from_operator(op, basis):
    m = np.zeros_like(op)
    for i, _ in np.ndenumerate(op):
        row, col = i
        t1 = np.dot(basis[row].conj().T, op @ basis[col])
        m[row, col] = t1
    return m


# Create a Hermitian operator matrix
op = np.array([[4, -2], [-2, 4]], dtype=complex)

# Get the eigenvalues and eigenvectors for the operator
eigen_vals, eigen_vecs = np.linalg.eig(op)

# Get the operator's components using its eigenvectors as its basis
m = get_matrix_from_operator(op, eigen_vecs)

display(array_to_latex(op, prefix=r"\mathbf{\hat{O}}="))
display(array_to_latex(eigen_vecs, prefix=r"\mathbf{\epsilon}="))
display(array_to_latex(m, prefix=r"\mathbf{O}="))
display(
    array_to_latex(
        eigen_vals[:, np.newaxis].T,
        prefix=r"\mathbf{\lambda}=",
    )
)

In [ ]:
# Cell 6 - Calculate the Commutator

n = 3
omega_1 = create_hermitian_matrix(n)
omega_2 = create_hermitian_matrix(n)

commutator = np.dot(omega_1, omega_2) - np.dot(omega_2, omega_1)

display(array_to_latex(omega_1, prefix=r"\mathbf{\Omega_1}="))
display(array_to_latex(omega_2, prefix=r"\mathbf{\Omega_2}="))

display(array_to_latex(commutator, prefix=r"\mathbf{[\Omega_1,\Omega_2]}="))
display(
    Math(
        rf"\mathbf{{\Omega_1}}\;\text{{and}}\;\mathbf{{\Omega_2}}\;"
        rf"\text{{commute}}\;?\;\rightarrow\;{np.isclose(commutator,0).all()}"
    )
)

In [ ]:
# Cell 7 - All diagonal matrices commute with each other


def create_diagonal_matrix(n):
    a = np.zeros((n, n), dtype=complex)
    for i in range(n):
        r1 = np.random.random()
        r2 = np.random.random()
        a[i, i] = complex(r1, r2)
    return a


omega_1 = create_diagonal_matrix(n)
omega_2 = create_diagonal_matrix(n)

commutator = np.dot(omega_1, omega_2) - np.dot(omega_2, omega_1)

display(array_to_latex(omega_1, prefix=r"\mathbf{\Omega_1}="))
display(array_to_latex(omega_2, prefix=r"\mathbf{\Omega_2}="))

display(array_to_latex(commutator, prefix=r"\mathbf{[\Omega_1,\Omega_2]}="))
display(
    Math(
        rf"\mathbf{{\Omega_1}}\;\text{{and}}\;\mathbf{{\Omega_2}}\;"
        rf"\text{{commute}}\;?\;\rightarrow\;{np.isclose(commutator,0).all()}"
    )
)